In [ ]:
import pandas as pd
import numpy as np
import numpy.random as npr
import random
import sparse
import json

from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, roc_auc_score, auc, average_precision_score

from missingness import get_adjusted_linear_model, compute_r, transform_Xs_to_Xt


In [ ]:
%load_ext autoreload
%autoreload 2

# Load in data from FIDDLE Data Preprocessing Pipeline

In [ ]:
data_folder = 'mimic-eicu-fiddle-feature/FIDDLE_eicu'
task = 'mortality_48h'

In [ ]:
# csv with patientunitstayid's and their corresponding hospital id's, for the top three hospitals with the most data
ids = pd.read_csv('top3_hosp_ids.csv')

# features
s = sparse.load_npz(f'{data_folder}/features/{task}/s.npz')  # time-invariant features
X = sparse.load_npz(f'{data_folder}/features/{task}/X.npz')  # time-dependent features
X = X.max(axis=1)  # compress to take the max value (note: the min is 0)

s_feature_names = json.load(open(f'{data_folder}/features/{task}/s.feature_names.json', 'r')) 
X_feature_names = json.load(open(f'{data_folder}/features/{task}/X.feature_names.json', 'r'))

# labels
df_pop = pd.read_csv(f'{data_folder}/population/{task}.csv')

In [ ]:
# extract part relevant to cohort
cohort_mask = df_pop['ID'].isin(ids['patientunitstayid'])

# fixed features
fixed_feats = s[cohort_mask].todense()
fixed_feats = pd.DataFrame(fixed_feats, columns=s_feature_names)

# in timed features, delete mask features
timed_feats = X[cohort_mask].todense()
mask_idxs = list([i for (i,f) in enumerate(X_feature_names) if 'mask' in f])
timed_feats = np.delete(timed_feats, mask_idxs, 1)
new_names = list([f for (i, f) in enumerate(X_feature_names) if i not in mask_idxs])
timed_feats = pd.DataFrame(timed_feats, columns=new_names)

features = pd.concat([fixed_feats, timed_feats], axis=1)
outcomes = df_pop[cohort_mask].set_index('ID')

outcomes = ids.merge(outcomes.reset_index(), left_on='patientunitstayid', right_on='ID').drop('patientunitstayid', axis=1)
features = pd.concat([outcomes[['ID', 'hospitalid']], features], axis=1)

features.to_csv(f'data/eicu/{task}_features.csv', index=None)
outcomes.to_csv(f'data/eicu/{task}_outcomes.csv', index=None)

# In future runs, comment out pre-processing and re-load preprocessed dataframe
# features = pd.read_csv(f'data/eicu/{task}_features.csv')
# outcomes = pd.read_csv(f'data/eicu/{task}_outcomes.csv')


In [ ]:
outcomes.groupby('hospitalid').count()

In [ ]:
# separate into data from each hospital
hospid1 = 73
hospid2 = 264
label_name = 'mortality_LABEL'

X1 = features[features['hospitalid'] == hospid1].drop('hospitalid', axis=1).set_index('ID')
X2 = features[features['hospitalid'] == hospid2].drop('hospitalid', axis=1).set_index('ID')

y1 = outcomes[outcomes['hospitalid'] == hospid1].drop('hospitalid', axis=1)[label_name]
y2 = outcomes[outcomes['hospitalid'] == hospid2].drop('hospitalid', axis=1)[label_name]

In [ ]:
# preprocessing: drop columns with few observations
props1 = X1.mean(axis=0).reset_index()
keep_cols1 = props1[props1[0] > 0.05]['index'].tolist()
drop_cols1 = props1[props1[0] < 0.01]['index'].tolist()

props2 = X2.mean(axis=0).reset_index()
keep_cols2 = props2[props2[0] > 0.05]['index'].tolist()
drop_cols2 = props2[props2[0] < 0.01]['index'].tolist()

drop_cols = list(set(drop_cols1 + drop_cols2))
keep_cols = list(set(keep_cols1 + keep_cols2))
keep_cols = list([c for c in keep_cols if c not in drop_cols])
print(len(keep_cols))
print(keep_cols)
print(X1.shape)
print(X2.shape)

In [ ]:
def sanity_check_stability(X_s, y_s):
    if isinstance(X_s, pd.DataFrame):
        X_s = X_s.values
        y_s = y_s.values[:, np.newaxis]
        
    reg = LinearRegression()
    reg.fit(X_s, y_s)
    preds = reg.predict(X_s)
    print('sklearn:')
    print(mean_squared_error(y_s, preds))
    print(roc_auc_score(y_s, (preds > 0.5).astype(int)))
    
    fn, beta_t = get_adjusted_linear_model(
        X_s, 
        y_s, 
        X_s, 
        fit_intercept=True, r=None, version='both')
    preds = fn(X_s)
    print('our implementation:')
    print(mean_squared_error(y_s, preds))
    print(roc_auc_score(y_s, (preds > 0.5).astype(int)))

In [ ]:
sanity_check_stability(X1[keep_cols], y1)
sanity_check_stability(X2[keep_cols], y2)

In [2]:
def mse_with_conf(pred, actual, var_scale=None):
    se = (pred - actual)**2
    mse = se.mean() 
    std = se.std() / np.sqrt(len(pred))
    if var_scale is not None:
        mse = mse / var_scale
        std = std / (var_scale**2)
    return mse, (mse - 1.69 * std, mse + 1.69 * std)

def get_bootstrap_metric(preds, actual, metric, replicates=1000):
    scores = []
    indices = list(range(len(preds)))
    for _ in range(replicates):
        sample = list(npr.choice(indices, size=len(preds), replace=True))
        preds_resampled = preds[sample]
        actual_resampled = actual[sample]
        score = metric(actual_resampled, preds_resampled)
        scores.append(score)
    mu = np.mean(scores)
    std = np.std(scores)
    return mu, (mu - 1.69 * std, mu + 1.69 * std)

def get_conf_str(v):
    med, (low, high) = v
    return f'{med:.3f} ({low:.3f} -- {high:.3f})'

def eval_nonparametric(
    X_s, y_s, rng,
    adjustment_X_t, eval_X_t, eval_y_t,
    description='', dname1=None, dname2=None,
    version='s', return_beta=False):
    
    if isinstance(X_s, pd.DataFrame):
        X_s = X_s.values
        adjustment_X_t = adjustment_X_t.values
        eval_X_t = eval_X_t.values
        y_s = y_s.values[:, np.newaxis]
        eval_y_t = eval_y_t.values[:, np.newaxis]
    
    new_Xt = transform_Xs_to_Xt(X_s, adjustment_X_t, rng, 
                                r=None, loose=True)
    
    # drop cols with 0 variance
    keep_cols = [i for i in range(new_Xt.shape[1]) if (new_Xt.std(axis=0) > 0.001)[i]]
    new_Xt = new_Xt[:, keep_cols]
    eval_X_t = eval_X_t[:, keep_cols]
    
    fn, beta = get_adjusted_linear_model(
        new_Xt, 
        y_s, 
        new_Xt, 
        fit_intercept=True, r=None, version='both')

    preds = fn(eval_X_t)
    eval_y_t = eval_y_t.ravel()

    mse = mse_with_conf(preds, eval_y_t) #, var_scale=y_t.std()**2)
    auc_val = get_bootstrap_metric(
        (preds > 0.5).astype(int), eval_y_t, roc_auc_score)
    
    auprc = get_bootstrap_metric(
        (preds > 0.5).astype(int), eval_y_t, average_precision_score)
    
    result = {
        'Model Class': description,
        'Source Domain': dname1,
        'Target Domain': dname2,
        'MSE': get_conf_str(mse),
        'AUC': get_conf_str(auc_val),
        'AUPRC': get_conf_str(auprc),
    }
    if return_beta:
        return result, beta
    return result



def eval_linear(X_s, y_s, 
                adjustment_X_t, eval_X_t, eval_y_t,
                description='', dname1=None, dname2=None,
                version='t', return_beta=False):
    if isinstance(X_s, pd.DataFrame):
        X_s = X_s.values
        adjustment_X_t = adjustment_X_t.values
        eval_X_t = eval_X_t.values
        y_s = y_s.values[:, np.newaxis]
        eval_y_t = eval_y_t.values[:, np.newaxis]
    fn, beta = get_adjusted_linear_model(X_s, y_s, adjustment_X_t, version=version)
    preds = fn(eval_X_t).ravel()
    eval_y_t = eval_y_t.ravel()

    mse = mse_with_conf(preds, eval_y_t) #, var_scale=y_t.std()**2)
    auroc = get_bootstrap_metric((preds / preds.max()), eval_y_t, roc_auc_score)
    auprc = get_bootstrap_metric((preds / preds.max()), eval_y_t, average_precision_score)

    result = {
        'Model Class': description,
        'Source Domain': dname1,
        'Target Domain': dname2,
        'MSE': get_conf_str(mse),
        'AUC': get_conf_str(auroc),
        'AUPRC': get_conf_str(auprc),
    }
    if return_beta:
        return result, beta
    return result

results = []

rng = npr.default_rng(0)
npr.seed(10)
random.seed(10)

X1, y1 = shuffle(X1, y1)
cutoff = int(0.8 * len(y1))
X1_tr, y1_tr = X1[:cutoff], y1[:cutoff]
X1_te, y1_te = X1[cutoff:], y1[cutoff:]

X2, y2 = shuffle(X2, y2)
cutoff = int(0.8 * len(y2))
X2_tr, y2_tr = X2[:cutoff], y2[:cutoff]
X2_te, y2_te = X2[cutoff:], y2[cutoff:]

# oracles
result = eval_linear(
    X1_tr[keep_cols], y1_tr, 
    X1_tr[keep_cols], X1_te[keep_cols], y1_te,
    description='oracle', dname1='Hospital 1', dname2='Hospital 1',
)
results.append(result)

result = eval_linear(
    X2_tr[keep_cols], y2_tr, 
    X2_tr[keep_cols], X2_te[keep_cols], y2_te,
    description='oracle', dname1='Hospital 2', dname2='Hospital 2',
)
results.append(result)

# not adjusted
result = eval_linear(
    X1[keep_cols], y1, 
    X1[keep_cols], X2[keep_cols], y2,
    description='unadjusted', dname1='Hospital 1', dname2='Hospital 2',
)
results.append(result)

result = eval_linear(
    X2[keep_cols], y2, 
    X2[keep_cols], X1[keep_cols], y1,
    description='unadjusted', dname1='Hospital 2', dname2='Hospital 1',
)
results.append(result)

# adjusted using t
result = eval_linear(
    X1[keep_cols], y1, 
    X2[keep_cols], X2[keep_cols], y2,
    description='adjusted using t', dname1='Hospital 1', dname2='Hospital 2',
)
results.append(result)

result = eval_linear(
    X2[keep_cols], y2, 
    X1[keep_cols], X1[keep_cols], y1,
    description='adjusted using t', dname1='Hospital 2', dname2='Hospital 1',
)
results.append(result)

# nonparametric adjustment
result = eval_nonparametric(
    X1[keep_cols], y1, rng,
    X2[keep_cols], X2[keep_cols], y2,
    description='nonparametric adjustment', dname1='Hospital 1', dname2='Hospital 2',
    version='n/a', return_beta=False)
results.append(result)

result = eval_nonparametric(
    X2[keep_cols], y2, rng,
    X1[keep_cols], X1[keep_cols], y1,
    description='nonparametric adjustment', dname1='Hospital 2', dname2='Hospital 1',
    version='n/a', return_beta=False)
results.append(result)


In [ ]:
print(pd.DataFrame(results).sort_values('Target Domain').to_latex(index=None))

# Print information about strongest coefficients for Hospital 1 and Hospital 2 Oracles

In [ ]:
result, beta = eval_linear(
    X1[keep_cols], y1, 
    X1[keep_cols], X1[keep_cols], y1,
    description='oracle', dname1='Hospital 1', dname2='Hospital 1',
    version='s', return_beta=True
)
strong_coefs1 = list(sorted(list(reversed(sorted(zip(beta.ravel().tolist(), keep_cols), key=lambda x: abs(x[0]))))[:5]))
strong_coefs1 = list([c for v, c in strong_coefs1])
coefmap1 = {c: v for v, c in zip(beta.ravel().tolist(), keep_cols)}

result, beta = eval_linear(
    X2[keep_cols], y2, 
    X2[keep_cols], X2[keep_cols], y2,
    description='oracle', dname1='Hospital 2', dname2='Hospital 2',
    version='s', return_beta=True
)
strong_coefs2 = list(sorted(list(reversed(sorted(zip(beta.ravel().tolist(), keep_cols), key=lambda x: abs(x[0]))))[:5]))
strong_coefs2 = list([c for v, c in strong_coefs2])
coefmap2 = {c: v for v, c in zip(beta.ravel().tolist(), keep_cols)}


In [ ]:
relative_nonzeros = pd.concat([X1[keep_cols].mean(axis=0), X2[keep_cols].mean(axis=0), X2[keep_cols].mean(axis=0) / X1[keep_cols].mean(axis=0)], axis=1)
top_coefs = relative_nonzeros[relative_nonzeros.index.isin(strong_coefs1 + strong_coefs2)]
top_coefs['Hospital 1 Oracle Coefficient'] = [coefmap1[v] for v in top_coefs.index.tolist()]
top_coefs['Hospital 2 Oracle Coefficient'] = [coefmap2[v] for v in top_coefs.index.tolist()]
top_coefs = top_coefs.rename({
    0: 'Hospital 1 Nonzero Proportion (p1)',
    1: 'Hospital 2 Nonzero Proportion (p2)',
    2: 'p2/p1 = 1 - r'
}, axis=1)
top_coefs = top_coefs[[
    'Hospital 1 Oracle Coefficient', 
    'Hospital 2 Oracle Coefficient',
    'Hospital 1 Nonzero Proportion (p1)',
    'Hospital 2 Nonzero Proportion (p2)', 
    'p2/p1 = 1 - r',
]]

In [ ]:
print(top_coefs.round(3).to_latex())